In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_util import *

from data_generator import DataGenerator

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models

import tensorflow as tf
#import tensorflow_addons as tfa

from tensorflow.keras.layers.experimental.preprocessing import Normalization


from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard
print(tf.__version__)

2.5.0-rc3


In [28]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)

METRICS = [
      #keras.metrics.TruePositives(name='truePositives'),
      #keras.metrics.FalsePositives(name='falsePositives'),
      #keras.metrics.TrueNegatives(name='trueNegatives'),
      #keras.metrics.FalseNegatives(name='falseNegatives'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [59]:
#Model 

def add_deep_layers(x, units):
    #x = Dropout(0.2)(x)
    x = Dense(units, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(units, activation='relu')(x)
    return x

def generate_model(seqs, features, dim, dropout, lays, lays_seq, train):
    reset_seed()
    inputX = Input(shape=features)
    
    normalizer.adapt(train)
    
    x = normalizer(inputX)
    x = inputX
    
    for lay in range(int(lays)):
        x = add_deep_layers(x, dim)
    
    x = Dense(10, activation='relu')(x)
    
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[inputX], outputs=x)
        
    return model


path = "./data/"


In [60]:


HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([60, 100, 500]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5]))
HP_LAY = hp.HParam('layers', hp.Discrete([1, 8, 20]))

#Seqs
HP_NUM_SEQS = hp.HParam('num_seqs', hp.Discrete([60]))
HP_LAY_SEQ = hp.HParam('layers_seq', hp.Discrete([0]))


METRIC_ACCURACY = 'precision'
METRIC_LOSS = 'loss'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_SEQS, HP_NUM_UNITS, HP_DROPOUT, HP_LAY_SEQ, HP_LAY ],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='precision')],
  )
  
trainX, trainY = load_data("", "train", path)
print("Loaded: {} {} ".format(trainX.shape, trainY.shape))
features = trainX.shape[-1]

normalizer = tf.keras.layers.experimental.preprocessing.Normalization()

def train_test_model(hparams):
    reset_seed()
    
    dim = hparams[HP_NUM_UNITS]
    seqs = hparams[HP_NUM_SEQS]
    dropout = hparams[HP_DROPOUT]
    lays = hparams[HP_LAY]
    lays_seq = hparams[HP_LAY_SEQ]

    #x, y = create_dataset(trainX, trainY, seqs, 0)
    
    x, y = get_balanced_set(trainX, trainY)
    
    model = generate_model(seqs = seqs,
                           features = features,
                           dim = dim,
                           dropout = dropout,
                           lays = lays,
                           lays_seq = lays_seq,
                           train = x
                          )
    
    #radam = tfa.optimizers.RectifiedAdam()
    #ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
    model.compile(
        #optimizer=ranger,
        optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.0001, rho=0.95, epsilon=1e-07),
        loss='binary_crossentropy',
        metrics=METRICS
    )
        
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_precision', 
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)

    callbacks_list = [early_stopping]
    
    history = model.fit(
        x,
        y,
        batch_size = 64,
        shuffle=True,
        validation_split = 0.03,
        #steps_per_epoch = len(train_generator),
        epochs=100,
        verbose=1,
        callbacks=callbacks_list
    )
    
    model_loaded = model
    
    def average(lst): 
        return sum(lst) / len(lst)
    
    def get_acc(db):
        print("evaluating {}".format(db))
        valX, valY = load_data(db, "val", path)
        evaluate = model_loaded.evaluate(valX, valY)
        print(evaluate)
        return evaluate[2]
    
    evals = []
    evals.append(get_acc(""))
    #evals.append(get_acc("btcusd"))
    #evals.append(get_acc("ethusd"))
    #evals.append(get_acc("ltcusd"))
    precision = average(evals)
    #print("precision: {}".format(precision))
    return precision

Loaded: (38413, 5) (38413,) 


In [61]:
best_hparams = {}
best_acc = 0

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        

In [62]:
session_num = 0

hparams_list = []

for num_seqs in HP_NUM_SEQS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in HP_DROPOUT.domain.values:
            for lay_seq in HP_LAY_SEQ.domain.values:
                for lay in HP_LAY.domain.values:
                    hparams = {
                    HP_NUM_SEQS: num_seqs,
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_LAY_SEQ: lay_seq,
                    HP_LAY: lay
                    }
                    hparams_list.append(hparams)

print("Total hparams: {}".format(len(hparams_list)))

!rm -Rf logs

for hparams in hparams_list:
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run_name = 'logs/hparam_tuning/' + run_name
    run(run_name, hparams)
    session_num += 1
    
print("Best acc {} hparams {}".format(best_acc, best_hparams))

Total hparams: 9
--- Starting trial: run-0
{'num_seqs': 60, 'num_units': 60, 'dropout': 0.5, 'layers_seq': 0, 'layers': 1}
Epoch 1/100
1052/1052 [==============================] - 3s 2ms/step - loss: 0.6948 - accuracy: 0.5035 - precision: 0.4850 - recall: 0.3967 - auc: 0.5013 - prc: 0.4884 - val_loss: 0.7244 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_prc: 1.0000
Epoch 2/100
1052/1052 [==============================] - 1s 1ms/step - loss: 0.6949 - accuracy: 0.5148 - precision: 0.4701 - recall: 0.0106 - auc: 0.4879 - prc: 0.4754 - val_loss: 0.7241 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_prc: 1.0000
Epoch 3/100
1052/1052 [==============================] - 1s 1ms/step - loss: 0.6946 - accuracy: 0.5152 - precision: 0.4895 - recall: 0.0114 - auc: 0.4932 - prc: 0.4805 - val_loss: 0.7238 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
#tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

In [ ]:


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = []

history = model.fit_generator(train_generator, 
                              validation_data=val_generator, 
                              validation_steps=len(val_generator)-1,
                              steps_per_epoch=len(train_generator)-1, 
                              epochs=20, verbose=1, 
                              callbacks=callbacks_list)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
import datetime
print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

In [ ]:
    inputX = Input(shape=(seqs, features))
    x = Dense(dim)(inputX)
    x = BatchNormalization()(x)
    x = LSTM(units=dim, return_sequences=True)(x)
    for lay in range(int(lays_seq)):
        x = add_deep_layers(x, dropout, dim)
    x = LSTM(dim, return_sequences=False)(x)
    
    for lay in range(int(lays)):
        x = add_deep_layers(x, dropout, dim)
        
    x = add_deep_layers(x, dropout, 20)
    x = Dense(10)(x)
    x = Dense(1, activation='sigmoid')(x)